In [1]:
import gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from collections import deque
import random
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'nes_py'

In [ ]:
env = gym_tetris.make('TetrisA-v0')
env = JoypadSpace(env, MOVEMENT)

done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

env.close()

In [6]:
state_size = (84, 84, 1)
action_size = env.action_space.n
memory = deque(maxlen=2000)
batch_size = 64
gamma = 0.95
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
target_update_counter = 0
update_target_every = 5
episodes = 1000
max_steps_per_episode = 1000
learning_rate = 0.001

In [7]:
def build_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=state_size))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate))
    return model

In [8]:
dqn = build_model()
target_dqn = build_model()

C:\Users\bsilver3\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [9]:
def preprocess_rgb(state):
    state = cv2.resize(state, (84, 110))
    state = cv2.cvtColor(state, cv2.COLOR_BGR2GRAY)
    state = state[26:110, :]
    state = state.astype(np.float32) / 255.0
    return state.reshape(84, 84, 1)

In [10]:
def preprocess_grayscale(state):
    if state is None:
        return np.zeros((84, 84, 1), dtype=np.float32)  # Return a blank image if state is None
    state = cv2.cvtColor(state, cv2.COLOR_RGB2GRAY) if state.shape[-1] == 3 else state  # Convert RGB to grayscale if needed
    state = cv2.resize(state, (84, 84))  # Resize the image
    state = state.astype(np.float32) / 255.0  # Normalize pixel values
    return np.expand_dims(state, axis=-1)  # Ensure the image has a single channel

In [11]:
# Take a random action
action = env.action_space.sample()

# Perform the action and capture the return values in a single variable
info = env.step(action)

info

C:\Users\bsilver3\AppData\Roaming\Python\Python39\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


(array([[[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [ 50, 132,  50],
         [ 50, 132,  50],
         [ 50, 132,  50]],
 
        ...,
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
  